# IA3微调

In [6]:
from peft import IA3Config,get_peft_model,TaskType
from transformers import AutoModelForCausalLM,AutoTokenizer,TrainingArguments,Trainer,default_data_collator
from datasets import load_dataset
import warnings
warnings.filterwarnings('ignore')

## 1、加载模型和分词器

In [7]:
model_name="Qwen/Qwen3-0.6B"
model=AutoModelForCausalLM.from_pretrained(model_name,device_map='auto',torch_dtype="auto")
tokenizer=AutoTokenizer.from_pretrained(model_name)

Loading weights: 100%|██████████| 311/311 [00:00<00:00, 462.26it/s, Materializing param=model.norm.weight]                              


In [8]:
model

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 1024)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=1024, out_features=2048, bias=False)
          (k_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (up_proj): Linear(in_features=1024, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((1024,), eps=1e-06)
        (post_attention_layer

## 2、IA3Config配置

In [9]:
config = IA3Config(
    task_type=TaskType.CAUSAL_LM,
    init_ia3_weights=True
)

In [10]:
def print_module_names(model, prefix=""):
    for name, module in model.named_children():
        full_name = f"{prefix}.{name}" if prefix else name
        print(full_name)
        print_module_names(module, full_name)

print_module_names(model)

model
model.embed_tokens
model.layers
model.layers.0
model.layers.0.self_attn
model.layers.0.self_attn.q_proj
model.layers.0.self_attn.k_proj
model.layers.0.self_attn.v_proj
model.layers.0.self_attn.o_proj
model.layers.0.self_attn.q_norm
model.layers.0.self_attn.k_norm
model.layers.0.mlp
model.layers.0.mlp.gate_proj
model.layers.0.mlp.up_proj
model.layers.0.mlp.down_proj
model.layers.0.mlp.act_fn
model.layers.0.input_layernorm
model.layers.0.post_attention_layernorm
model.layers.1
model.layers.1.self_attn
model.layers.1.self_attn.q_proj
model.layers.1.self_attn.k_proj
model.layers.1.self_attn.v_proj
model.layers.1.self_attn.o_proj
model.layers.1.self_attn.q_norm
model.layers.1.self_attn.k_norm
model.layers.1.mlp
model.layers.1.mlp.gate_proj
model.layers.1.mlp.up_proj
model.layers.1.mlp.down_proj
model.layers.1.mlp.act_fn
model.layers.1.input_layernorm
model.layers.1.post_attention_layernorm
model.layers.2
model.layers.2.self_attn
model.layers.2.self_attn.q_proj
model.layers.2.self_attn

In [11]:
model = get_peft_model(model, config)
model.print_trainable_parameters()  # 打印可训练参数

trainable params: 172,032 || all params: 751,804,416 || trainable%: 0.0229


## 3、加载数据集

In [12]:
data=load_dataset('json',data_files='../../dataset/chinese_law_ft_dataset.json',split="train[:1000]")

In [13]:
dataset=data.train_test_split(
    train_size=0.7,
    shuffle=True,
    seed=7
)

In [14]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'id'],
        num_rows: 700
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'id'],
        num_rows: 300
    })
})

## 4、数据预处理

In [15]:
def process_fun(example):
    content=[]
    for instruction,input,output in zip(example['instruction'],example['input'],example['output']):
        if input.strip():
            text=f'Human:{instruction}\n{input}\nAI:{output}'
            content.append(text)
        else:
            text=f'Human:{instruction}\nAI:{output}'
            content.append(text)
    
    encoded = tokenizer(
        content,
        max_length=512,
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    labels = encoded["input_ids"].clone()
    for index,text in enumerate(content):
        answer_start=text.find('AI:')+len('AI:')
        question=text[:answer_start]
        question_ids=tokenizer.encode(question, add_special_tokens=False)
        question_length=len(question_ids)
        labels[index,:question_length]=-100
    return {
        "input_ids": encoded["input_ids"],
        "attention_mask": encoded["attention_mask"],
        "labels": labels
    }


In [16]:
train_process_data=dataset['train'].map(process_fun,batched=True,remove_columns=dataset['train'].column_names)
test_process_data=dataset['test'].map(process_fun,batched=True,remove_columns=dataset['test'].column_names)

## 5、模型训练配置

In [17]:
# 定义训练参数
training_args = TrainingArguments(
    output_dir="../../models/IA3_tuning",
    logging_steps=10,
    logging_dir='./runs',
    eval_strategy='epoch',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=1e-4,
    weight_decay=0.01,
    metric_for_best_model='eval_loss',
    gradient_accumulation_steps=4,  # 如果GPU内存有限
    fp16=True
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


## 6、训练模型

In [18]:
trainer=Trainer(
    model=model,
    args=training_args,
    eval_dataset=test_process_data,
    train_dataset=train_process_data,
    data_collator=default_data_collator,
)

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,11.649712,11.329892
2,10.861726,10.759994
3,10.530606,10.563810


TrainOutput(global_step=132, training_loss=11.207109248999393, metrics={'train_runtime': 8632.5423, 'train_samples_per_second': 0.243, 'train_steps_per_second': 0.015, 'total_flos': 3846347056742400.0, 'train_loss': 11.207109248999393, 'epoch': 3.0})

## 7、保存模型

In [20]:
trainer.save_model('../../models/IA3_tuning')